In [1]:
import psycopg2
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from IPython.display import display, display_markdown
 

# connect to the database
connection = psycopg2.connect(user="postgres",
                                    password="postgres",
                                    host="127.0.0.1",
                                    port="5432",
                                    database="postgres")

def sql_to_matplotlibTable(conn, query, column_names, title):
    cursor = conn.cursor()
    try:
      cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
      print("Error: %s" % error)
      cursor.close()
      return 1
    # The execute returns a list of tuples:
    tuples_list = cursor.fetchall()
    cursor.close()
    # Now we need to transform the list into a pandas DataFrame:
    df = pd.DataFrame(tuples_list, columns=column_names)
    display_markdown("# "+ title, raw=True)
    display(df)
    return df


def get_openehr_conditions():
    # Define the API endpoint URL
    url = 'https://openehr-api.dev.carebase24.io/rest/openehr/v1'
    listTemplatesUrl = '/definition/template/adl1.4'
    aql = '/query/aql'

    try:
        session = requests.Session()
        username = 'myuser'
        password = 'myPassword432'
        # Make a GET request to the API endpoint using requests.get()
        headers = { 'Content-Type' : 'application/json',
            'Accept': 'application/json',
            'Prefer' : 'return=minimal'}
        aql_query = {"q" : "SELECT c/content[openEHR-EHR-EVALUATION.problem_diagnosis.v1]/data[at0001]/items[at0002]/value/value, e/ehr_id.value FROM EHR e CONTAINS COMPOSITION c[openEHR-EHR-COMPOSITION.report.v1]"}
        response = requests.post(url+aql, auth=HTTPBasicAuth(username, password), headers=headers, json=aql_query)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            posts = response.json()
            patients = []
            condition_values = []
            conditions= posts["rows"]
            for condition in conditions:
                patients.append(condition[1]["value"])
                condition_values.append(condition[0])
            #print(patients)       
            #print(condition_values)
            df = pd.DataFrame({'person_source_value': patients, 'condition': condition_values})
            #print(df)
            return df     
        
        else:
            print('Error:', response.content)
            return None
    except requests.exceptions.RequestException as e:
        # Handle any network-related errors or exceptions
        print('Error:', e)
        return None
  
# SHOW PERSON DATA
select_query_person = """SELECT person_id, person_source_value FROM person"""
columns=['person_id', 'person_source_value']
dfPerson = sql_to_matplotlibTable(connection, select_query_person, columns, 'OMOP Person Table')

# SHOW CONDITION OCCURRENCE DATA

select_query_condition_occurrence = """SELECT person_id, condition_source_value, condition_concept_id  FROM condition_occurrence WHERE person_id = 58"""
columns=['person_id', 'condition', 'condition_concept_id']
dfConditions = sql_to_matplotlibTable(connection, select_query_condition_occurrence, columns, 'OMOP Condition Occurrence Table')

dfConditions = dfConditions.merge(dfPerson, on='person_id', how='left')
#print(dfConditions)

# 709044004	46271022
# 1	58	44054006

update_query_condition_occurrence = """update condition_occurrence set condition_source_value = 'Diabetes Mellitus Type 2' WHERE person_id = 58 and condition_source_value = '709044004' """
# columns=['person_id', 'condition', 'condition_concept_id']
dfConditions_updated = sql_to_matplotlibTable(connection, update_query_condition_occurrence, columns, 'OMOP Condition Occurrence Table - Update Keetje')

# dfOpenEhrCondition = get_openehr_conditions()
# print(dfOpenEhrCondition)

# pk_columns = ['person_source_value', 'condition']
# # dfOpenEhrCondition = dfOpenEhrCondition.merge(dfConditions, on=pk_columns, how='left')
# dfOpenEhrCondition = dfOpenEhrCondition.merge(dfConditions, on=['person_id'], how='left')
# dfOpenEhrCondition.rename(columns={'person_source_value': 'patient_id'}, inplace=True)
# dfOpenEhrCondition.rename(columns={'condition': 'open_ehr_condition_code'}, inplace=True)
# dfOpenEhrCondition.rename(columns={'condition_concept_id': 'omop_condition_concept_id'}, inplace=True)
# dfOpenEhrCondition = dfOpenEhrCondition.drop(columns=['person_id'])

# display_markdown("# Condition OpenEHR/OMOP", raw=True)
# display(dfOpenEhrCondition)



# OMOP Person Table

,person_id,person_source_value
0,1,None
1,2,None
2,3,None
3,4,None
4,5,None
5,6,None
6,7,None
7,8,None
8,9,None
9,10,None


# OMOP Condition Occurrence Table

,person_id,condition,condition_concept_id
0,58,709044004,46271022
1,58,44054006,201826


ProgrammingError: no results to fetch

In [ ]:
# SHOW PERSON DATA
select_query_person = """SELECT person_id, person_source_value FROM person"""
columns=['person_id', 'person_source_value']
dfPerson = sql_to_matplotlibTable(connection, select_query_person, columns, 'OMOP Person Table')
# dfPerson.columns

In [4]:
dictio = {'person_id': [58, 58], 'condition_code': [709044004, 44054006], 'condition_display': ['Diabetes Mellitus Type 2', 'Broken'],'condition_concept_id': [46271022,201826 ]}
df = pd.DataFrame.from_dict(dictio)

In [5]:
df.head()


,person_id,condition_code,condition_display,condition_concept_id
0,58,709044004,Diabetes Mellitus Type 2,46271022
1,58,44054006,Broken,201826
